In [2]:
import pandas as pd
import numpy as np



**Cargar bases de datos limpias**

*Cargamos las bases de datos que la api consumirá para cada una de nuestras funciones. Estas bases de datos se trabajaron externamente para que fueran consumidas por la api de forma correcta.* 


In [86]:
df_1= pd.read_parquet(r'C:\Users\Lenovo\Desktop\Proyecto Individual\BD Funciones\endpoint_1.parquet')

In [82]:
df_2= pd.read_parquet(r'C:\Users\Lenovo\Desktop\Proyecto Individual\BD Funciones\endpoint_2.parquet')

In [83]:
df_3= pd.read_parquet(r"C:\Users\Lenovo\Desktop\Proyecto Individual\BD Funciones\endpoint_3.parquet")

In [93]:
df_4= pd.read_parquet(r"C:\Users\Lenovo\Desktop\Proyecto Individual\BD Funciones\endpoint_4_5.parquet")

In [84]:
df_5= pd.read_parquet(r"C:\Users\Lenovo\Desktop\Proyecto Individual\BD Funciones\endpoint_4_5.parquet")

*La base de datos endpoint_4_5.parquet, la utilizaremos para las funciones best_developer_year y developer_reviews_analysis*

In [89]:
df_2

,user_id,item_id,recommend,item_name,price
0,76561197970982479,1250,True,Killing Floor,19.99
1,76561197970982479,22200,True,Zeno Clash,9.99
2,js41637,227300,True,Euro Truck Simulator,19.99
3,js41637,239030,True,Papers Please,9.99
4,evcentric,370360,True,TIS,6.99
...,...,...,...,...,...
37252,76561198239215706,730,True,CounterStrike Global Offensive,14.99
37253,wayfeng,730,True,CounterStrike Global Offensive,14.99
37254,76561198251004808,253980,True,Enclave,4.99
37255,72947282842,730,True,CounterStrike Global Offensive,14.99


# EndPoint 1

*Necesitamos crear nuestro primer endpoint, que corresponde a una función developer(desarrollador: str) que devuelve la cantidad de items y porcentaje de contenido FREE por año según empresa desarrolladora.* 

In [94]:
def developer(developer: str, df: pd.DataFrame):
    ''' devuelve la cantidad de items y porcentaje de contenido
      FREE por año según empresa desarrolladora.
      Parámetros:
      developer: nombre de desarrollador'''
    df_filtrado = df_1[df_1['developer'] == developer]
    
    if df_filtrado.empty:
        return f"No se encontraron datos para el developer: {developer}"
    
    # Agrupa por año y cuenta la cantidad de juegos por año
    conteo_apps = df_filtrado.groupby('year')['app_name'].count().reset_index()
    conteo_apps = conteo_apps.rename(columns={'app_name': 'Cantidad de Ítems'})

    # Calcula el porcentaje de aplicaciones gratuitas (price = 0)
    gratuitas = df_filtrado[df_filtrado['price'] == 0].groupby('year')['app_name'].count()
    total_por_año = df_filtrado.groupby('year')['app_name'].count()
    porcentaje_gratis = (gratuitas / total_por_año * 100).reset_index(name='Contenido Free')

    # Combina los resultados en un solo DataFrame
    resultado = pd.merge(conteo_apps, porcentaje_gratis, on='year', how='left').fillna(0)
    resultado['Contenido Free'] = resultado['Contenido Free'].astype(int).astype(str) + '%'

    return resultado

# Chequear 
dev = 'Dovetail Games'  
developer(df_1, dev)



ValueError: Operands are not aligned. Do `left, right = left.align(right, axis=1, copy=False)` before operating.

# Endpoint_2

*Función userdata (User_id : str) Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews recommend y la cantidad de items*

In [38]:
def userdata(df, user_id):
    # Se filtran los datos para el usuario dado
    user_data = df[df['user_id'] ==user_id]

    if user_data.empty:
        return {"Error": f"No se encontraron datos para el usuario: {user_id}"}

    # Calcula la cantidad de dinero gastado
    total_spent = user_data['price'].sum()

    # Calcula el porcentaje de recomendaciones (True vs. total de reviews)
    recommend_percentage = (user_data['recommend'].mean()) * 100

    # Calcula la cantidad de ítems comprados o revisados
    total_items = user_data['item_id'].nunique()

    # Retorna los resultados en un diccionario
    return {
        "User_id": user_id,
        "Total_gastado": round(total_spent, 2),
        "%_recomendacion": round(recommend_percentage, 2),
        "Cantidad_items": total_items
    }

# Ejemplo de uso
result = userdata(df_merged,'76561197970982479')
print(result)

{'User_id': '76561197970982479', 'Total_gastado': 29.98, '%_recomendacion': 100.0, 'Cantidad_items': 2}


# EndPoint 3

*Función uUserForGenre (genero: str) Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

In [167]:
def UserForGenre(df, genero):
    # Se filtra el DataFrame por el género seleccionado
    genre_data = df[df['genres'] == genero]

    # Se calcula las horas de juego por usuario (user_id)
    horas_por_usuario = genre_data.groupby('user_id')['playtime_forever_hours'].sum()

    # Se encuentra el 'user_id' con más horas jugadas en este género
    max_jugador = horas_por_usuario.idxmax()

    # Se filtran los datos solo para el jugador con más horas
    datos_jugador = genre_data[genre_data['user_id'] == max_jugador]

    # Agrupar por año y sumar las horas jugadas por año
    horas_por_año = datos_jugador.groupby('year')['playtime_forever_hours'].sum().reset_index()

    # Convertir el resultado a una lista de diccionarios
    max_usuario_año_playtime_list = [
        {"Año": int(year), "Horas": float(hours)} for year, hours in zip(horas_por_año['year'], horas_por_año['playtime_forever_hours'].round(1))
    ]

    # Retornar los resultados como un diccionario
    return {
        "Género": genero,
        "Usuario": max_jugador,
        "Horas_jugadas_por_año": max_usuario_año_playtime_list
    }

 

In [168]:
genero='Action'
UserForGenre(df_endpoint3, genero)

{'Género': 'Action',
 'Usuario': 'laerte261',
 'Horas_jugadas_por_año': [{'Año': 2012, 'Horas': 0.5},
  {'Año': 2013, 'Horas': 4836.3}]}

# Endpoint 4

In [80]:
def top3_developers_by_year(year: int, df: pd.DataFrame):
    """
    Esta función devuelve una lista de diccionarios con los 3 desarrolladores 
    con más juegos recomendados por los usuarios para un año específico.

    Parámetros:
    - year: Año para el cual se desea obtener el top 3 de desarrolladores.
    - df: DataFrame con los datos de juegos.

    Retorno:
    - Lista de diccionarios en el formato [{'Puesto 1': X}, {'Puesto 2': Y}, {'Puesto 3': Z}].
    """
    # Filtrar el DataFrame por el año proporcionado
    df_year = df[df['year'] == year]

    # Filtrar los juegos que fueron recomendados (true_recommend == True)
    df_recommended = df_year[df_year['true_recommend'] == True]

    # Contar la cantidad de juegos recomendados por cada desarrollador
    top_developers = (
        df_recommended.groupby('developer')['item_name']
        .nunique()  # Contar juegos únicos recomendados por cada developer
        .sort_values(ascending=False)  # Ordenar de mayor a menor
        .head(3)  # Obtener los 3 primeros
    )

    # Crear la lista de diccionarios con los puestos
    resultado = [
        {f"Puesto {i + 1}": developer}
        for i, developer in enumerate(top_developers.index)
    ]

    return resultado
top3_developers_by_year(2014,df_merged_5)

[{'Puesto 1': 'Valve'},
 {'Puesto 2': 'Team Digital Ltd'},
 {'Puesto 3': 'Double Fine Productions'}]

# EndPoint 5

In [71]:
def developer_reviews_analysis(desarrollador: str, df: pd.DataFrame):
    """
    Devuelve un diccionario con el nombre del desarrollador como llave y 
    una lista con el total de reseñas categorizadas como negativas y positivas.
    
    Parámetros:
    - desarrollador: Nombre del desarrollador a analizar.
    - df: DataFrame con los datos de reseñas y análisis de sentimiento.

    Retorno:
    - Diccionario en el formato {'Developer': [Negative = X, Positive = Y]}.
    """

    # Filtrar los datos por el desarrollador proporcionado
    developer_df = df[df['developer'] == desarrollador]

    if developer_df.empty:
        return {desarrollador: 'No hay registros para este desarrollador'}

    # Contar las reseñas negativas (asumimos que 'sentiment_analysis' == 0 es negativo)
    negative_count = developer_df[developer_df['sentiment_analysis'] == 0].shape[0]

    # Contar las reseñas positivas (asumimos que 'sentiment_analysis' == 1 es positivo)
    positive_count = developer_df[developer_df['sentiment_analysis'] == 1].shape[0]

    # Crear el diccionario con los resultados
    resultado = {
        desarrollador: [f"Negative = {negative_count}", f"Positive = {positive_count}"]
    }

    return resultado

In [77]:
developer_reviews_analysis('Valve', df_merged_5)

{'Valve': ['Negative = 438', 'Positive = 165']}